In [1]:
!pip install -q imageio
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs

In [2]:
!pip install tensorflow_hub

  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)


In [4]:
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed

logging.set_verbosity(logging.ERROR)

# some modules to help with reading the UCF101 dataset.
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np

# some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request

In [25]:
# utilities to fetch videos from UCF101 dataset
UCF_ROOT = 'https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/'
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()
unverified_context = ssl._create_unverified_context()

In [41]:
# get the kinetics-400 action labels from the GitHub repository.
KINETICS_URL = 'https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt'

with request.urlopen(KINETICS_URL) as obj:
    labels = [line.decode("utf-8").strip() for line in obj.readlines()]

print('Found %d labels.' % len(labels))

Found 400 labels.


In [42]:
def list_ucf_videos():
    
    """Lists videos available in UCF101 dataset."""
    global _VIDEO_LIST
    if not _VIDEO_LIST:
        index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode('utf-8')
        videos = re.findall('(v_[\w_]+\.avi)', index)
        _VIDEO_LIST = sorted(set(videos))
    
    return list(_VIDEO_LIST)

In [43]:
def fetch_ucf_video(video):
    """Fetchs a video and cache into local filesystem."""
    cache_path = os.path.join(_CACHE_DIR, video)
    if not os.path.exists(cache_path):
        urlpath = request.urljoin(UCF_ROOT, video)
        print('Fetching %s => %s' % (urlpath, cache_path))
        data = request.urlopen(urlpath, context=unverified_context).read()
        open(cache_path, 'wb').write(data)
    return cache_path

In [44]:
# utilities to open video files using CV2
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:(start_y + min_dim), start_x: (start_x + min_dim)]

In [45]:
def load_video(path, max_frames=0, resize=(224, 224)):
    
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
      
            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    
    return np.array(frames) / 255.0

In [55]:
def to_gif(images):
    
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./animation.gif', converted_images, fps=25)
    
    embed.embed_file('./animation.gif')

In [56]:
def categorize_ucf_videos():
    
    ucf_videos = list_ucf_videos()
  
    categories = {}
    for video in ucf_videos:
        category = video[2:-12]
        if category not in categories:
            categories[category] = []
        categories[category].append(video)

    print('Found %d videos in %d categories.' % (len(ucf_videos), len(categories)))

    #for category, sequences in categories.items():
    #    summary = ", ".join(sequences[:2])
    #    print("%-20s %4d videos (%s, ...)" % (category, len(sequences), summary))
    
    return categories

In [57]:
def load_sample_videos():
    
    videos = ['v_SoccerPenalty_g01_c02.avi', 'v_CricketShot_g04_c02.avi', 
              'v_PlayingGuitar_g01_c02.avi', 'v_HorseRace_g01_c01.avi']
    
    sample_videos = []
    for video in videos:
        video_path = fetch_ucf_video(video)
        sample_video = load_video(video_path)
        sample_videos.append(sample_video)
    
    #print(sample_videos.shape)
    
    return videos, sample_videos

In [58]:
def load_model():

    i3d = hub.load('https://tfhub.dev/deepmind/i3d-kinetics-400/1').signatures['default']
    return i3d

In [59]:
def predict(model, video_name, sample_video):
  
    # Add a batch axis to the sample video.
    model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

    logits = model(model_input)['default'][0]
    probabilities = tf.nn.softmax(logits)

    print('Top 5 actions for {}:'.format(video_name))
    for i in np.argsort(probabilities)[::-1][:5]:
        print(f'  {labels[i]:22}: {probabilities[i] * 100:5.2f}%')

In [60]:
def main():
    
    categorize_ucf_videos()
    videos, sample_videos = load_sample_videos()
    i3d = load_model()
    
    for index, sample_video in enumerate(sample_videos):
        to_gif(sample_video)
        predict(i3d, videos[index], sample_video)

In [61]:
if __name__ == '__main__':
    main()

Found 13320 videos in 101 categories.
Top 5 actions for v_SoccerPenalty_g01_c02.avi:
  shooting goal (soccer): 69.46%
  marching              : 14.34%
  kicking field goal    :  4.03%
  playing tennis        :  3.56%
  passing American football (in game):  2.60%
Top 5 actions for v_CricketShot_g04_c02.avi:
  playing cricket       : 97.77%
  skateboarding         :  0.71%
  robot dancing         :  0.56%
  roller skating        :  0.56%
  golf putting          :  0.13%
Top 5 actions for v_PlayingGuitar_g01_c02.avi:
  playing guitar        : 89.51%
  strumming guitar      :  9.17%
  busking               :  0.35%
  playing drums         :  0.28%
  recording music       :  0.23%
Top 5 actions for v_HorseRace_g01_c01.avi:
  marching              : 88.00%
  shaking hands         :  1.98%
  shredding paper       :  1.49%
  driving car           :  1.18%
  kicking field goal    :  1.02%
